In [1]:
import pandas as pd

In [2]:
chatbotData = pd.read_csv('data/ChatData.csv')
question, answer = list(chatbotData['Q']) , list(chatbotData['A'])

len(answer)

11823

In [3]:
# test로 100개만
question = question[:100]
answer = answer[:100]

for i in range(100) :
    print('질문:',question[i])
    print('답변:',answer[i])
    print(' ')

질문: 12시 땡!
답변: 하루가 또 가네요.
 
질문: 1지망 학교 떨어졌어
답변: 위로해 드립니다.
 
질문: 3박4일 놀러가고 싶다
답변: 여행은 언제나 좋죠.
 
질문: 3박4일 정도 놀러가고 싶다
답변: 여행은 언제나 좋죠.
 
질문: PPL 심하네
답변: 눈살이 찌푸려지죠.
 
질문: SD카드 망가졌어
답변: 다시 새로 사는 게 마음 편해요.
 
질문: SD카드 안돼
답변: 다시 새로 사는 게 마음 편해요.
 
질문: SNS 맞팔 왜 안하지ㅠㅠ
답변: 잘 모르고 있을 수도 있어요.
 
질문: SNS 시간낭비인 거 아는데 매일 하는 중
답변: 시간을 정하고 해보세요.
 
질문: SNS 시간낭비인데 자꾸 보게됨
답변: 시간을 정하고 해보세요.
 
질문: SNS보면 나만 빼고 다 행복해보여
답변: 자랑하는 자리니까요.
 
질문: 가끔 궁금해
답변: 그 사람도 그럴 거예요.
 
질문: 가끔 뭐하는지 궁금해
답변: 그 사람도 그럴 거예요.
 
질문: 가끔은 혼자인게 좋다
답변: 혼자를 즐기세요.
 
질문: 가난한 자의 설움
답변: 돈은 다시 들어올 거예요.
 
질문: 가만 있어도 땀난다
답변: 땀을 식혀주세요.
 
질문: 가상화폐 쫄딱 망함
답변: 어서 잊고 새출발 하세요.
 
질문: 가스불 켜고 나갔어
답변: 빨리 집에 돌아가서 끄고 나오세요.
 
질문: 가스불 켜놓고 나온거 같아
답변: 빨리 집에 돌아가서 끄고 나오세요.
 
질문: 가스비 너무 많이 나왔다.
답변: 다음 달에는 더 절약해봐요.
 
질문: 가스비 비싼데 감기 걸리겠어
답변: 따뜻하게 사세요!
 
질문: 가스비 장난 아님
답변: 다음 달에는 더 절약해봐요.
 
질문: 가장 확실한 건 뭘까?
답변: 가장 확실한 시간은 오늘이에요. 어제와 내일을 놓고 고민하느라 시간을 낭비하지 마세요.
 
질문: 가족 여행 가기로 했어
답변: 온 가족이 모두 마음에 드는 곳으로 가보세요.
 
질문: 가족 여행 고고
답변: 온 가족이 모두 마음에 드는 곳으로 가보세요.
 
질문: 가족 여행 어디로 가지?
답변: 

In [4]:
import re
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
from konlpy.tag import Okt
from keras import models, layers, optimizers, preprocessing, metrics

Using TensorFlow backend.


In [5]:
# 태그단어
PAD = "<PADDING>"  # 패딩
STA = "<START>"    # 시작
END = "<END>"      # 끝
OOV = "<OOV>"      # Out Of Vocabulary

PAD_INDEX  = 0
STA_INDEX  = 1
END_INDEX  = 2
OOV_INDEX  = 3

ENCODER_INPUT = 0
DECODER_INPUT = 1
DECODER_TARGET = 2

# 한 문장에서 단어 시퀀스의 최대 개수
maxSequences = 30

# 임베딩 벡터 차원
embeddingDim = 100

# LSTM 출력 차원
lstmHiddenDim = 128

# 정규표현식 필터
RE_FILTER = re.compile("[.,!?\"':;~()]")

In [6]:
# 형태소 분석기
def posTag(sentences) : 
    
    tagger = Okt()
    sentencePos = []
    
    for sentence in sentences :
        # 특수문자 제거
        sentence = re.sub(RE_FILTER, '', sentence)
        sentence = ' '.join(tagger.morphs(sentence))
        sentencePos.append(sentence)
        
    return sentencePos

question = posTag(question)
answer = posTag(answer)

In [7]:
# 질문 + 대답 문장을 하나로 합치기
sentences = []
sentences.extend(question)
sentences.extend(answer)
len(sentences)  # 200

# 단어배열 생성
words = []

for sentence in sentences :
    for word in sentence.split() :
        words.append(word)
len(words) # 966개

# words에서 길이가 0인 단어를 삭제
words = [w for w in words if len(w)>0]
len(words)  # 966개

# 중복단어 삭제
words = list(set(words))
len(words)  # 450개

# 리스트 맨 앞에 추가
words[:0] = [PAD, STA, END, OOV]

# 인덱스 부여해 딕셔너리 생성
wordToIndex = dict(zip(words, range(len(words))))
indexToWord = dict(zip(range(len(words)),words ))

# 전처리
# 문장 => 인덱스
def convertTextToIndex(sentences, voc, mytype) : 
    
    sentencesIndex = []
    
    for sentence in sentences :
        sentenceIndex = []
        if mytype == DECODER_INPUT : 
            sentenceIndex.extend([voc[STA]])
        for word in sentence.split() :
            if voc.get(word) :
                sentenceIndex.extend([voc.get(word)])
            else :
                sentenceIndex.extend([voc.get(OOV)])
                
        if mytype == DECODER_TARGET : 
            # 디코더 출력은 맨 마지막에 <END>가 추가
            if maxSequences <= len(sentenceIndex) :
                sentenceIndex = sentenceIndex[:maxSequences-1] + [voc[END]]
            else :
                sentenceIndex += [voc[END]]
        
        elif len(sentenceIndex) > maxSequences :
            sentenceIndex = sentenceIndex[:maxSequences]
        # 패딩
        sentenceIndex += [wordToIndex[PAD]] * (maxSequences-len(sentenceIndex))
        
        # 결과값 저장
        sentencesIndex.append(sentenceIndex)
    
    return np.asarray(sentencesIndex)

# 인코더 입력, 디코더 입력 출력 -> 인덱스 변환
xEncoder = convertTextToIndex(question, wordToIndex, ENCODER_INPUT)
xDecoder = convertTextToIndex(answer, wordToIndex, DECODER_INPUT)
yDecoder = convertTextToIndex(answer, wordToIndex, DECODER_TARGET)

In [8]:
print(xEncoder[0])  # 12시 땡
print(xDecoder[0])  # START 하루 가 또 가네요
print(yDecoder[0])  #       하루 가 또 가네요 END

[309 258   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0]
[  1 269 267 375 142   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0]
[269 267 375 142   2   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0]


In [9]:
oneHotData = np.zeros([len(yDecoder), maxSequences, len(words)])
# 100(답변 개수), 30(최대 단어 개수), 454(전체 단어 집합 개수)

In [10]:
np.shape(oneHotData)

for i, seq in enumerate(yDecoder) : 
    for j, index in enumerate(seq) :
        oneHotData[i,j,index] = 1
        
yDecoder = oneHotData

yDecoder[0].shape  # 첫번째 답변 내용의 shape

(30, 454)

In [39]:
#훈련 모델 생성
#인코더 정의

#입력 문장의 인덱스 sequence를 입력
encoderInputs=layers.Input(shape=(None,))
#임베딩 계층
encoderOutputs=layers.Embedding(len(words),embeddingDim)(encoderInputs)

encoderOutputs,stateH, stateC=layers.LSTM(lstmHiddenDim,return_state=True, 
            dropout=0.2, recurrent_dropout=0.5)(encoderOutputs)
#return_state=True => 상태값 리턴
#LSTM은 2개 상태 존재(셀, 히든 스테이트)

encoderStates=[stateH, stateC]


#디코더 정의
#출력 문장의 인덱스 sequence를 입력
decoderInputs=layers.Input(shape=(None,))
#임베딩 계층
decoderEmbedding=layers.Embedding(len(words),
                                embeddingDim)
decoderOutputs=decoderEmbedding(decoderInputs)



decoderLSTM=layers.LSTM(lstmHiddenDim,
                        return_state=True, 
            return_sequences=True, 
                        dropout=0.2, 
                        recurrent_dropout=0.5)
decoderOutputs, _, _=decoderLSTM(decoderOutputs,initial_state=encoderStates)
decoderDense=layers.Dense(len(words), 
                          activation="softmax")
decoderOutputs=decoderDense(decoderOutputs)


In [40]:
model=models.Model([encoderInputs, decoderInputs], decoderOutputs)
model.compile(optimizer='rmsprop',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [41]:
#예측 모델 인코더 정의
encoderModel=models.Model(encoderInputs, 
                          encoderStates)

#예측 모델 디코더 정의
#바로 앞에 있는 디코더의 출력(상태)을 입력 받아서
#예측을 해야 함.
decoderStateInputH=layers.Input(shape=(lstmHiddenDim,))
decoderStateInputC=layers.Input(shape=(lstmHiddenDim,))
decoderStatesInputs=[decoderStateInputH,decoderStateInputC]

#임베딩 계층
decoderOutputs=decoderEmbedding(decoderInputs)
#LSTM 계층
decoderOutputs, stateH, stateC=decoderLSTM(decoderOutputs,
           initial_state=decoderStatesInputs)
decoderStates=[stateH, stateC]

#Dense계층을 통해 원핫 형식으로 예측 단어 인덱스를 추출
decoderOutputs=decoderDense(decoderOutputs)

#예측 모델 디코더 설정
decoderModel=models.Model([decoderInputs]+decoderStatesInputs,
            [decoderOutputs]+decoderStates)

In [42]:
#인덱스를 문장으로 변환
def convertIndexToText(indexs, voc):
    #구현
    sentence=""
    for i in indexs:
        if i==END_INDEX: #종료 인덱스
            break;
        if voc.get(i) is not None:
            sentence+=voc[i]
        else:
            sentence.extend([voc[OOV_INDEX]])
        sentence +=" "    
    return sentence    

In [43]:
#에폭
for epoch in range(10):
    print("total epoch:", epoch+1)
    history=model.fit([xEncoder,xDecoder], yDecoder,
             epochs=100,
             batch_size=64,
                     verbose=0)
    print("accuracy :", history.history['accuracy'])
    print("loss :", history.history['loss'])
    #문장 예측
    #3박 4일 놀러 가고 싶다 -> 여행 은 언제나 좋죠
    
    inputEncoder=xEncoder[2].reshape(1,xEncoder[2].shape[0]) #(30,) ->(1,30)
    inputDecoder=xDecoder[2].reshape(1,xDecoder[2].shape[0]) #(30,) ->(1,30)
    
    results=model.predict([inputEncoder,inputDecoder])
    
    #결과값에 대해서 가장 큰 값의 위치를 구함
    index=np.argmax(results[0], 1)
    #인덱스 -> 문장으로 변환
    sentence=convertIndexToText(index, indexToWord)
    print(sentence)
    print()
    

total epoch: 1
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

accuracy : [0.29233333, 0.7863333, 0.7863333, 0.7863333, 0.7863333, 0.7863333, 0.7863333, 0.7863333, 0.7863333, 0.7863333, 0.7863333, 0.7863333, 0.7863333, 0.7863333, 0.7863333, 0.7876667, 0.7876667, 0.79, 0.78866667, 0.788, 0.789, 0.79066664, 0.789, 0.79366666, 0.78966665, 0.79366666, 0.7953333, 0.79466665, 0.79366666, 0.79433334, 0.7963333, 0.79433334, 0.795, 0.794, 0.799, 0.801, 0.806, 0.8006667, 0.8016667, 0.8013333, 0.808, 0.79933333, 0.8056667, 0.812, 0.80866665, 0.809, 0.8103333, 0.81233335, 0.81166667, 0.8146667, 0.8146667, 0.81633335, 0.818, 0.818, 0.8196667, 0.818, 0.8093333, 0.817, 0.8193333, 0.822, 0.8206667, 0.82266665, 0.821, 0.82166666, 0.8233333, 0.8206667, 0.82166666, 0.8236667, 0.824, 0.82266665, 0.825, 0.824, 0.8243333, 0.8243333, 0.8236667, 0.82566667, 0.825, 0.82533336, 0.825, 0.8243333, 0.827, 0.82633334, 0.827, 0.82633334, 0.827, 0.827, 0.82633334, 0.8276

loss : [0.43334063053131106, 0.43531235456466677, 0.43173054814338685, 0.42730132818222044, 0.4258242034912109, 0.420757315158844, 0.4150869607925415, 0.4152486526966095, 0.41067564964294434, 0.4080134046077728, 0.40471389770507815, 0.4039977777004242, 0.40138347625732423, 0.39462866067886354, 0.3974061918258667, 0.39015034794807435, 0.3888171756267548, 0.38666136026382447, 0.38506449341773985, 0.3850569450855255, 0.3769657027721405, 0.37584728479385376, 0.37524752259254457, 0.3685429906845093, 0.3676715123653412, 0.36560667634010313, 0.3627638363838196, 0.3613349676132202, 0.3608634889125824, 0.3553044188022614, 0.35423237562179566, 0.35126739621162417, 0.3482010352611542, 0.3434231042861938, 0.3427443826198578, 0.340367511510849, 0.3366564166545868, 0.3365026879310608, 0.3352054965496063, 0.3309874677658081, 0.3271376371383667, 0.32459925770759585, 0.32138580560684205, 0.32023552775382996, 0.31729950070381163, 0.3155402433872223, 0.3151473665237427, 0.3118582785129547, 0.311600056886

loss : [0.13135862588882447, 0.13091281294822693, 0.1290402317047119, 0.12846723437309265, 0.12920089364051818, 0.12862383723258972, 0.1272703665494919, 0.13102307617664338, 0.12634713113307952, 0.1265801763534546, 0.12501057118177414, 0.12598664700984955, 0.12608492851257325, 0.12393198162317276, 0.12361862361431122, 0.12406382113695144, 0.1245300105214119, 0.1218913596868515, 0.12324459493160247, 0.12471661269664765, 0.12361293733119964, 0.12176873207092286, 0.12107043623924256, 0.12192294090986251, 0.12052086263895034, 0.12107653975486755, 0.11866935551166534, 0.11971447438001633, 0.11912356436252594, 0.11981566220521928, 0.11913132071495056, 0.11801077306270599, 0.11842733144760131, 0.12215593695640564, 0.11831804007291793, 0.11723364561796189, 0.1183241993188858, 0.11720724612474441, 0.11600228041410446, 0.11585677862167358, 0.11583658993244171, 0.11642139554023742, 0.1158345451951027, 0.11507994681596756, 0.11571388870477677, 0.11539908349514008, 0.11388063222169877, 0.1146367460

loss : [0.0952977529168129, 0.09581425487995147, 0.09501499205827713, 0.09579060345888138, 0.09433328688144683, 0.09529747635126114, 0.09660229831933975, 0.09788424491882325, 0.09646736055612565, 0.09542937695980072, 0.09575795948505401, 0.09370966523885726, 0.0951089996099472, 0.09426077663898468, 0.09439306020736694, 0.09502733021974563, 0.09349779576063157, 0.09311080366373062, 0.09422494500875472, 0.09443188905715942, 0.09296837121248246, 0.09322465807199479, 0.09557798266410827, 0.09292139649391175, 0.09403116464614868, 0.09374043107032776, 0.09378198862075805, 0.0939660620689392, 0.09480705261230468, 0.09272617906332016, 0.09308503925800324, 0.09226472496986389, 0.09232019871473313, 0.09214458405971528, 0.09390623092651368, 0.09264579564332961, 0.09379966676235199, 0.09274610221385955, 0.092256338596344, 0.09165597319602967, 0.09219387620687484, 0.09176857531070709, 0.09227164506912232, 0.09161036610603332, 0.09355926781892776, 0.09100004583597184, 0.0922402560710907, 0.092367085

loss : [0.0818703556060791, 0.08277353256940842, 0.08287375837564469, 0.08372716963291169, 0.08182830452919006, 0.08251799374818802, 0.08176595568656922, 0.08167889535427093, 0.08287277579307556, 0.08346825629472733, 0.08250896126031876, 0.08109280675649642, 0.08221127569675446, 0.08213051795959472, 0.08308053970336914, 0.08063996523618698, 0.08208282262086869, 0.08291386812925339, 0.08525343626737594, 0.08311809301376342, 0.0810684123635292, 0.08045109063386917, 0.08251080811023712, 0.08172654151916504, 0.08102274656295777, 0.08333559691905976, 0.0818823117017746, 0.08061903297901153, 0.08146471738815307, 0.07991370290517807, 0.08201370298862458, 0.08273517668247223, 0.0817640569806099, 0.08164283782243728, 0.08222512483596801, 0.08110116094350815, 0.0833959323167801, 0.08160387247800827, 0.0800059700012207, 0.08021575331687927, 0.081016925573349, 0.08026597291231155, 0.08120562195777893, 0.0813360235095024, 0.08072669953107833, 0.08059525609016419, 0.07913056403398513, 0.080529063940

In [50]:
# 문장 입력
def makePredictInput (sentence) :
    
    sentences = []
    sentences.append(sentence)
    
    sentences = posTag(sentences)
    
    inputSeq = convertTextToIndex(sentences, wordToIndex, ENCODER_INPUT)
    
    return inputSeq

In [55]:
# 예측 답변 생성
def genereateText(inputSeq) :
    # 입력을 인코더에 넣고, 마지막 상태 구함
    states = encoderModel.predict(inputSeq)
    
    # 목표 시퀀스 초기화
    targetSeq = np.zeros((1,1))
    targetSeq[0,0] = STA_INDEX  # <START>시그널 인덱스 추가
    # 인덱스 초기화
    indexs = []
    
    # 디코더 반복
    while 1 :
        decoderOutputs, stateH, stateC = decoderModel.predict([targetSeq] + states)
        # 결과를 원핫인코딩 형식으로 변환
        index = np.argmax(decoderOutputs[0,0,:])
        indexs.append(index)
        
        # 종료 체크
        if index == END_INDEX or len(indexs)>=maxSequences :
            break
        
        # targetSeq를 이전 출력으로 설정
        targetSeq = np.zeros((1,1))
        targetSeq[0,0] = index  

        states = [stateH, stateC]
    
    # 인덱스를 문장으로 변환
    sentence = convertIndexToText(indexs, indexToWord)
    return sentence

In [61]:
inputSeq = makePredictInput('3박 4일 놀러가고 싶지않다')
# [[320,157,...,19,0,0,0...,0]]
inputSeq

array([[415, 227,  11, 285,   4,   3,   3,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0]])

In [62]:
sentence = genereateText(inputSeq)
sentence

'여행 은 언제나 좋죠 '

In [26]:

# 모델 생성
## 인코더 정의
# 입력 문장의 인덱스 sequence를 입력
encoderInputs = layers.Input(shape=(None, ))
# 임베딩 계층          454           100
encoderOutputs = layers.Embedding(len(words), embeddingDim)(encoderInputs)
# LSTM
encoderOutputs, stateH, stateC = layers.LSTM(lstmHiddenDim, return_state=True,return_sequences=False, dropout=0.2, recurrent_dropout=0.5)(encoderOutputs)
# return_state = True => 상태값 리턴  / LSTM은 2개의 상태 존재(셀, 히든스테이트)
encoderStates = [stateH, stateC]
## 디코더 정의
# 출력 문장의 인덱스 sequence를 입력
decoderInputs = layers.Input(shape=(None, ))
# 임베딩 계층                           454           100
decoderEmbedding = layers.Embedding(len(words),embeddingDim)
decoderOutputs = decoderEmbedding(decoderInputs)
# LSTM
decoderLSTM = layers.LSTM(lstmHiddenDim, return_state=True,return_sequences=True, dropout=0.2, recurrent_dropout=0.5)
decoderOutputs, _, _ = decoderLSTM(decoderOutputs, initial_state=encoderStates)
decoderDense = layers.Dense(len(words), activation='softmax')(decoderOutputs)

model = models.Model([encoderInputs, decoderInputs] ,decoderOutputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [27]:
## 예측 모델 인코더 정의
encoderModel= models.Model(encoderInputs, encoderStates)

## 예측 모델 디코더 정의
decoderStateInputH = layers.Input(shape=(lstmHiddenDim,))
decoderStateInputC = layers.Input(shape=(lstmHiddenDim,))
decoderStateInputs = [decoderStateInputH, decoderStateInputC]

# 임베딩 계층
decoderOutputs = decoderEmbedding(decoderInputs)

# LSTM 계층
decoderOutputs, stateH, stateC = decoderLSTM(decoderOutputs, initial_state=decoderStateInputs)

decoderStates = [stateH, stateC]

# Dense계층을 통해 원핫 형식으로 예측 단어인덱스 추출
decoderOutputs = decoderDense(decoderOutputs)

# 예측모델 디코더 설정
model = models.Model([decoderInputs]+ decoderStateInputs, [decoderOutputs]+decoderStates)

TypeError: 'Tensor' object is not callable

In [29]:
# Train Model

############### 인코더 모델 정의
# 입력 문장의 인덱스 sequence를 입력
encoderInputs = layers.Input(shape=(None,))
# 임베딩 계층
encoderOutputs = layers.Embedding(len(words),embeddingDim)(encoderInputs)
# LSTM cell
encoderOutputs , stateH , stateC = layers.LSTM(lstmHiddenDim,return_state=True,return_sequences=False,dropout=0.2,recurrent_dropout=0.5)(encoderOutputs)
encoderStates = [stateH , stateC]

############### 디코더 모델 정의
# 출력 문장의 인덱스 sequence를 입력
decoderInputs = layers.Input(shape=(None,))
# 임베딩 계층                           454           100
decoderEmbedding = layers.Embedding(len(words),embeddingDim)
decoderOutputs = decoderEmbedding(decoderInputs)
# LSTM cell
decoderLSTM = layers.LSTM(lstmHiddenDim,return_state=True,return_sequences=True,dropout=0.2,recurrent_dropout=0.5)
decoderOutputs , _ , _ = decoderLSTM(decoderOutputs,initial_state=encoderStates)

decoderDense = layers.Dense(len(words),activation='softmax')
decoderOutputs = decoderDense(decoderOutputs)

In [30]:
# 예측_Test Model

############### 인코더 모델 정의
encoderModel = models.Model(encoderInputs,encoderStates)

############### 디코더 모델 정의 : 바로 앞에 있는 디코더의 출력(상태)을 입력받아 예측해야함
decoderStateInputH = layers.Input(shape=(lstmHiddenDim,))
decoderStateInputC = layers.Input(shape=(lstmHiddenDim,))
decoderStateInputs = [decoderStateInputH , decoderStateInputC]
# 디코더 임베딩 계층
decoderOutputs = decoderEmbedding(decoderInputs)
# LSTM Cell
decoderOutputs , stateH , stateC = decoderLSTM(decoderOutputs,initial_state=decoderStateInputs)
decoderStates = [stateH , stateC]
# Dense 계층을 통해 원핫 형식으로 예측 단어 인덱스 추출
decoderOutputs = decoderDense(decoderOutputs)

# 예측 모델 디코더 설정
decoderModel = models.Model(inputs=[decoderInputs]+decoderStateInputs , outputs=[decoderOutputs]+decoderStates)

In [31]:
# 인덱스 문장으로 변환
def convertIndexToText(indexs, voc) :
    # 구현
    sentence = ''
    
    for i in indexs :
        if i == END_INDEX : # 종료 인덱스
            break
        if voc.get(i) is not None :
            sentence += voc[i]
        else :
            sentence.extend([voc[OOV_INDEX]])
        sentence += ' '
    return sentence


In [38]:
from keras_tqdm import tqdm_notebook_callback

for epoch in range(20) :
    print('total epoch : ', epoch+1)
    history = model.fit([xEncoder, xDecoder], yDecoder, epochs=100, batch_size=64)         
    print('accuracy : ', history.history['accuracy'])
    print('loss : ', history.history['loss'])
    
    # 문장예측
    # 3박 4일 놀러 가고 싶다 =? 여행 은 언제나 좋죠
    
    inputEncoder = xEncoder[2].reshape(1, xEncoder[2].shape[0])
    inputDecoder = xDecoder[2].reshape(1, xDecoder[2].shape[0])
    
    results = model.predict([inputEncoder, inputDecoder])
    
    # 결과값에 대해서 가장 큰 값으 위치를 구함
    index = np.argmax(results[0], 1)
    # 인덱스 -> 문장으로 변환
    sentence = convertIndexToText(index, indexToWord)
    print(sentence)
    print('')

total epoch :  1


ValueError: Error when checking target: expected lstm_4 to have shape (None, 128) but got array with shape (30, 454)

In [20]:
print()

In [ ]:
import MeCab
me = MeCab.Tagger()
res = me.parse('드라이브할 때 듣기 좋은 노래 들려줘')
print(res)